# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#Solving error where gmaps module was not found
import sys
!{sys.executable} -m pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Renaming the file in here
WeatherPyCities = "cities.csv"

#Loading the csv into a dataframe
VacationPyDf = pd.read_csv(WeatherPyCities)

#Dropping blanks
VacationPyDf.drop('Unnamed: 0', inplace=True, axis=1)

#Show figure
VacationPyDf



,City,Cloudiness,Country,Date,Humidity,Latitude,Max Temp,Wind Speed,longitude
0,bac lieu,56,VN,1674492867,83,9.2850,77.29,14.56,105.7244
1,hobart,32,AU,1674492867,93,-42.8794,60.26,1.14,147.3294
2,chokurdakh,100,RU,1674492867,93,70.6333,-24.97,6.11,147.9167
3,matay,0,EG,1674492390,50,28.4190,63.48,10.60,30.7792
4,jamestown,100,US,1674492510,75,42.0970,29.79,10.36,-79.2353
...,...,...,...,...,...,...,...,...,...
581,kalmunai,98,LK,1674492992,87,7.4167,76.42,7.70,81.8167
582,shelburne,93,CA,1674492992,94,44.0787,30.76,9.69,-80.2041
583,togul,100,RU,1674492993,95,53.4650,12.02,10.20,85.9128
584,luang prabang,5,LA,1674492993,58,19.8856,62.85,0.72,102.1347


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configuring gmaps
gmaps.configure(api_key=g_key)

#Setting lat and lng as locations
Locations = VacationPyDf[['Latitude', 'longitude']]

#Setting humidity as weight
Humidity = VacationPyDf['Humidity']

#Plotting the heatmap
heatmaps = gmaps.figure()

#Setting Size
heatmaps = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

#Creating the heatmap layer
heat_layer = gmaps.heatmap_layer(Locations, weights=Humidity)

#Adding the heatmap layer
heatmaps.add_layer(heat_layer)

#Showing figure
heatmaps



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#A max temperature lower than 80 degrees but higher than 70.
MaxTempdf = VacationPyDf[(VacationPyDf["Max Temp"] > 70) & (VacationPyDf["Max Temp"] < 80)]

#Wind speed less than 10 mph.
WindSpeeddf = MaxTempdf[MaxTempdf["Wind Speed"] < 10]

#Zero cloudiness.
ZeroClouddf = WindSpeeddf[WindSpeeddf["Cloudiness"] == 0]

# Drop any rows will null values:
IdealWeatherConditions = ZeroClouddf.dropna()
IdealWeatherConditions

,City,Cloudiness,Country,Date,Humidity,Latitude,Max Temp,Wind Speed,longitude
47,hilo,0,US,1674492873,75,19.7297,74.28,4.00,-155.0900
145,najran,0,SA,1674492891,24,17.4924,72.25,4.23,44.1277
456,pisco,0,PE,1674492970,57,-13.7000,78.85,6.91,-76.2167
468,mecca,0,SA,1674492881,60,21.4267,76.46,7.18,39.8261


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Storing into a variable
hotel_df = IdealWeatherConditions

#Adding Hotel name column
hotel_df['Hotel Name'] = ''

#Printing result to see error
hotel_df


,City,Cloudiness,Country,Date,Humidity,Latitude,Max Temp,Wind Speed,longitude,Hotel Name
47,hilo,0,US,1674492873,75,19.7297,74.28,4.00,-155.0900,
145,najran,0,SA,1674492891,24,17.4924,72.25,4.23,44.1277,
456,pisco,0,PE,1674492970,57,-13.7000,78.85,6.91,-76.2167,
468,mecca,0,SA,1674492881,60,21.4267,76.46,7.18,39.8261,


In [7]:
#Setting parameters, had changed hotel/hotels to lodging as results was not coming up for hotel.
params = {
     'radius': 5000,
     'types': 'lodging',
     'key': g_key,
}

for index, row in hotel_df.iterrows():
    Lat = row['Latitude']
    Lng = row['longitude']
    CityName = row['City']

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    params['location'] = f'{Lat},{Lng}'
    print(f'Retrieving Results for Index {index} - {CityName}')
    response = requests.get(base_url, params=params)
    response = response.json()

    try:
     
     print(f'Closest hotel in the city {CityName} is {response["results"][0]["name"]}.')
     hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
     print('-'*30)

    except:
          print('Unsuccessful')



Retrieving Results for Index 47 - hilo
Closest hotel in the city hilo is Hilo Hawaiian Hotel.
------------------------------
Retrieving Results for Index 145 - najran
Closest hotel in the city najran is محمد هادي ال فطيح.
------------------------------
Retrieving Results for Index 456 - pisco
Closest hotel in the city pisco is Hotel San Isidro Oficial.
------------------------------
Retrieving Results for Index 468 - mecca
Closest hotel in the city mecca is Makkah Clock Royal Tower, A Fairmont Hotel.
------------------------------


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "longitude"]]

In [9]:
#Setting Size
heatmaps = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

#Layer codes
HeatLayer = gmaps.heatmap_layer(Locations, weights=Humidity)
MarkerLayer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#Adding Layers
heatmaps.add_layer(HeatLayer)
heatmaps.add_layer(MarkerLayer)

#Show figure
heatmaps

Figure(layout=FigureLayout(height='420px'))